# Colisiones

## Billar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
rc('animation', html='jshtml')
rc('figure',figsize=(4,3))

In [ ]:
z = np.array([[x,y] for y in [0.8, 0.6] for x in np.linspace(0.2,0.8,4)])
n = z.size//2

z[0] = 0.1,0.1

v = 0*np.random.rand(n,2)
v[0] = (1,3)


R = 0.04*np.ones([n,1]) + 0.02*np.linspace(0,1,n).reshape(-1,1)

R[0]=0.05

m = (R**3).flatten()

colores = ['b', 'g', 'r', 'c', 'm', 'y', 'k','orange']


def choque(j,k,f=1):
    global v
    g = z[j]-z[k]
    n = g/np.linalg.norm(g)

    wj = n@v[j]
    VJ = v[j] - wj*n

    wk = n@v[k]
    VK = v[k] - wk*n

    A = m[j]*wj + m[k]*wk
    M = m[j] + m[k]
    d = wk-wj

    nj = f*(A + m[k]*d) / M
    nk = f*(A - m[j]*d) / M

    v[j] = nj*n + VJ
    v[k] = nk*n + VK



fps = 30   # imágenes por segundo
T   = 1    # tiempo total de simulación


dt = 0.01

fig, ax = plt.subplots(figsize=(4,4))
plt.axis('off');
plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)


dr = [plt.Circle(z[k],R[k],color=colores[k%len(colores)]) for k in range(n)]

for c in dr:
    ax.add_patch(c)

def animate(k):
    global z, v
    zs = z + v*dt
    reb = (zs > 1-R) |  (zs < R)
    v[reb] *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)

    z += v*dt

    for k in range(n):
        dr[k].center=z[k]

    return []

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

## Distribución de Maxwell - Boltzmann

La [distribución de la energía](https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution#Distribution_for_the_energy) es una chi-cuadrado cuya [distribución empírica](https://en.wikipedia.org/wiki/Chi-squared_distribution#Cumulative_distribution_function) en 2D tiene una forma analítica muy simple.

In [ ]:
z = np.array([[x,y] for y in [0.7, 0.8, 0.9] for x in np.linspace(0.1,0.9,10)])
n = z.size//2

z[0] = 0.1,0.1

v = 0*np.random.rand(n,2)
v[0] = (1,3)


R = 0.02 + 0.01*np.linspace(0,1,n).reshape(-1,1)

R[0]=0.04

m = (R**3).flatten()

Et = m[0]*(v[0]@v[0])/2
Em = Et/n

colores = ['b', 'g', 'r', 'c', 'm', 'y', 'k','orange']



dt = 0.01

fig = plt.figure(figsize=(10,5))
fig.suptitle('Maxwell-Boltzmann distribution')

ax = fig.add_axes([0,0,0.5,1])
ax.set_xlim(-0.1,1.1)
ax.set_ylim(-0.1,1.1)
ax.set_axis_off();
ax2 = fig.add_axes([0.55,0.25,0.4,0.5])
ax2.set_title('Energy CDF')

plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)

cdf, = ax2.plot([],[],color='red')
e = np.linspace(0,8,200)
ax2.plot(e/10,1-np.exp(-e/2),color='gray',ls='dashed')

dr = [plt.Circle(z[k],R[k],color=colores[k%len(colores)]) for k in range(n)]

for c in dr:
    ax.add_patch(c)

def animate(k):
    global z, v
    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1


    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)

    z += v*dt

    for k in range(n):
        dr[k].center=z[k]

    E = np.sum(v**2,axis=1)/2 * m

    cdf.set_data(sorted(E/Em/5),np.linspace(0,1,n))

    return []



fps = 30
T   = 1

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

## Distribución de velocidades

In [ ]:
from scipy.stats import norm, chi2, chi

z = np.array([[x,y] for y in [0.6, 0.7, 0.8, 0.9] for x in np.linspace(0.1,0.9,10)])
n = z.size//2

v = np.zeros([n,2])
v[:] = (1,0)

v[0] += 0.001*np.random.randn(2) 


R = 0.02*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

vdist = chi(2,scale=np.sqrt(Em/m[0]))

print('Em = {}, Vrms = {} = {}'.format(Em,np.sqrt(2*Em/m[0]), np.sqrt(vdist.moment(2))))

print('sigma = {}'.format(np.sqrt(Em/m[0])))

colores = ['darkgreen']


fig = plt.figure(figsize=(10,5))
fig.suptitle('2D Maxwell-Boltzmann distribution')

ax = fig.add_axes([0,0,0.5,1])
ax.set_xlim(-0.1,1.1)
ax.set_ylim(-0.1,1.1)
ax.set_axis_off();

ax2 = fig.add_axes([0.55,0.1,0.4,0.35])
ax2.set_title('Energy CDF')

ax3 = fig.add_axes([0.55,0.55,0.4,0.35])
ax3.set_title('Speed CDF')

plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)

cdf, = ax2.plot([],[],color='red')
e = np.linspace(0,5,200)
ax2.plot(e,1-np.exp(-e),color='gray',ls='dashed')

vcdf, = ax3.plot([],[],color='blue')
ax3.set_xlim(0,3)
ax3.set_ylim(-0.05,1.05)
vv = np.linspace(0,5,200);

ax3.plot(vv,chi(2,scale=np.sqrt(Em/m[0])).cdf(vv),color='gray',ls='dashed');


dr = [plt.Circle(z[k],R[k],color=colores[k%len(colores)]) for k in range(n)]

for c in dr:
    ax.add_patch(c)


dt = 0.01


def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)

    z += v*dt


    for k in range(n):
        dr[k].center=z[k]

    E = np.sum(v**2,axis=1)/2 * m

    cdf.set_data(sorted(E/Em),np.linspace(1/n,1,n))

    #print(v[:,0].mean(),v[:,0].std())
    V = np.sqrt(np.sum(v*v,axis=1))
    #print(V.mean(), 4*Em/m[0], vdist.mean())

    vcdf.set_data(sorted(V),np.linspace(1/n,1,n))

    return []


fps = 30
T   = 1

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

In [ ]:
from scipy.stats import norm, chi2, chi

z = np.array([[x,y] for y in [0.6, 0.7, 0.8, 0.9] for x in np.linspace(0.1,0.9,10)])
n = z.size//2

v = np.zeros([n,2])
v[:] = (1,0)

v[0] += 0.001*np.random.randn(2) 


R = 0.02*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

vdist = chi(2,scale=np.sqrt(Em/m[0]))

print('Em = {}, Vrms = {} = {}'.format(Em,np.sqrt(2*Em/m[0]), np.sqrt(vdist.moment(2))))

sigma = np.sqrt(Em/m[0])

print('sigma = {}'.format(sigma))

colores = ['darkgreen']


fig = plt.figure(figsize=(10,5))
fig.suptitle('2D Maxwell-Boltzmann distribution')

ax = fig.add_axes([0,0,0.5,1])
ax.set_xlim(-0.1,1.1)
ax.set_ylim(-0.1,1.1)
ax.set_axis_off();

ax2 = fig.add_axes([0.55,0.1,0.4,0.35])
ax2.set_title('$v_x$ CDF')

ax3 = fig.add_axes([0.55,0.55,0.4,0.35])
ax3.set_title('Speed CDF')

plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)

cdf, = ax2.plot([],[],color='red')
e = np.linspace(-2,2,200)
ax2.plot(e,norm(scale=sigma).cdf(e),color='gray',ls='dashed')

vcdf, = ax3.plot([],[],color='blue')
ax3.set_xlim(0,3)
ax3.set_ylim(-0.05,1.05)
vv = np.linspace(0,5,200);

ax3.plot(vv,chi(2,scale=np.sqrt(Em/m[0])).cdf(vv),color='gray',ls='dashed');


dr = [plt.Circle(z[k],R[k],color=colores[k%len(colores)]) for k in range(n)]

for c in dr:
    ax.add_patch(c)


dt = 0.01

def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)


    z += v*dt


    for k in range(n):
        dr[k].center=z[k]


    cdf.set_data(sorted(v[:,0]),np.linspace(1/n,1,n))

    V = np.sqrt(np.sum(v*v,axis=1))
    vcdf.set_data(sorted(V),np.linspace(1/n,1,n))

    return []


fps = 30
T   = 1

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

## 3D velocidad

In [ ]:
u = np.linspace(0.0, 2.0 * np.pi, 8)
v = np.linspace(0.0, np.pi, 8)
x_0 = np.outer(np.cos(u), np.sin(v))
y_0 = np.outer(np.sin(u), np.sin(v))
z_0 = np.outer(np.ones_like(u), np.cos(v))

def esfera(ax,pos,r):
    x,y,z = pos
    ax.plot_surface(x+x_0*r,y+y_0*r,z+z_0*r, cmap='gray');

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from scipy.stats import norm, chi2, chi

z = np.array([[x,y,z] for y in [0.6, 0.7, 0.8, 0.9] 
                        for x in np.linspace(0.1,0.9,10)
                        for z in [0.7,0.8,0.9]])
n = z.size//3

v = np.zeros([n,3])
v[:] = (1,0,0)

v[:] += 0.01*np.random.randn(n,3) 


R = 0.03*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_axes([0,0,1,1], projection='3d')
ax.set_xlim(0,1); ax.set_ylim(0,1); ax.set_zlim(0,1)
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
ax.view_init(azim=-40-110)

for p in z:  esfera(ax,p,0.03)

In [ ]:
dt = 0.01

def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)


    z += v*dt

    ax.clear()
    ax.set_xlim(0,1); ax.set_ylim(0,1); ax.set_zlim(0,1)
    ax.view_init(azim=-150+k*45/300)

    for p,r in zip (z,R):
        esfera(ax,p,r)

    return []


fps = 30
T   = 1
animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

In [ ]:
from scipy.stats import norm, chi2, chi

z = np.array([[x,y,z] for y in [0.6, 0.7, 0.8, 0.9] 
                        for x in np.linspace(0.1,0.9,10)
                        for z in [0.25,0.5,0.75]])
n = z.size//3

v = np.zeros([n,3])
v[:] = (1,0,0)

v[:] += 0.2*np.random.randn(n,3) 


R = 0.02*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

In [ ]:
vdist = chi(2,scale=np.sqrt(Em/m[0]))

print('Em = {}, Vrms = {} = {}'.format(Em,np.sqrt(2*Em/m[0]), np.sqrt(vdist.moment(2))))

sigma = np.sqrt(2/3*Em/m[0])

print('sigma = {}'.format(sigma))

colores = ['darkgreen']


fig = plt.figure(figsize=(10,5))
fig.suptitle('3D Maxwell-Boltzmann distribution')

ax = fig.add_axes([0,0,0.5,1])
ax.set_xlim(-0.1,1.1)
ax.set_ylim(-0.1,1.1)
ax.set_axis_off();

ax2 = fig.add_axes([0.55,0.1,0.4,0.35])
ax2.set_title('$v_x$ CDF')

ax3 = fig.add_axes([0.55,0.55,0.4,0.35])
ax3.set_title('Speed CDF')

#plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)

cdf, = ax2.plot([],[],color='red')
e = np.linspace(-2,2,200)
ax2.plot(e,norm(scale=sigma).cdf(e),color='gray',ls='dashed')
ax2.plot(e,norm(scale=np.sqrt(Em/m[0])).cdf(e),color='gray',lw=0.5)

vcdf, = ax3.plot([],[],color='blue')
ax3.set_xlim(0,3)
ax3.set_ylim(-0.05,1.05)
vv = np.linspace(0,5,200);

ax3.plot(vv,chi(2,scale=np.sqrt(Em/m[0])).cdf(vv),color='gray',lw=0.5);
ax3.plot(vv,chi(3,scale=sigma).cdf(vv),color='gray',ls='dashed');


dr = [plt.Circle(z[k][:2],R[k],color=colores[k%len(colores)], alpha=0.5) for k in range(n)]

for c in dr:
    ax.add_patch(c)

In [ ]:
dt = 0.01

def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)


    z += v*dt


    for k in range(n):
        dr[k].center=z[k][:2]
        dr[k].set_radius(0.01 + 0.03*z[k][2])


    cdf.set_data(sorted(v[:,0]),np.linspace(1/n,1,n))

    V = np.sqrt(np.sum(v*v,axis=1))
    vcdf.set_data(sorted(V),np.linspace(1/n,1,n))

    return []


fps = 30
T   = 1

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

## 3D energía

Comparando con 2D.

In [ ]:
from scipy.stats import norm, chi2, chi

z = np.array([[x,y,z] for y in [0.6, 0.7, 0.8, 0.9] 
                        for x in np.linspace(0.1,0.9,10)
                        for z in [0.25,0.5,0.75]])
n = z.size//3

v = np.zeros([n,3])
v[:] = (1,0,0)

v[:] += 0.2*np.random.randn(n,3) 


R = 0.02*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

In [ ]:
vdist = chi(2,scale=np.sqrt(Em/m[0]))

print('Em = {}, Vrms = {} = {}'.format(Em,np.sqrt(2*Em/m[0]), np.sqrt(vdist.moment(2))))

sigma = np.sqrt(2/3*Em/m[0])

print('sigma = {}'.format(sigma))

colores = ['darkgreen']


fig = plt.figure(figsize=(10,5))
fig.suptitle('3D Maxwell-Boltzmann distribution')

ax = fig.add_axes([0,0,0.5,1])
ax.set_xlim(-0.1,1.1)
ax.set_ylim(-0.1,1.1)
ax.set_axis_off();

ax2 = fig.add_axes([0.55,0.1,0.4,0.8])
ax2.set_title('Energy CDF')

#ax3 = fig.add_axes([0.55,0.55,0.4,0.35])
#ax3.set_title('Speed CDF')

#plt.close();

ax.plot([1,0,0,1,1],[0,0,1,1,0],'-',color='black',lw=3)

cdf, = ax2.plot([],[],color='red')
e = np.linspace(0,5,200)
ax2.plot(e,chi2(3,scale=1/3).cdf(e),color='gray',ls='dashed')
ax2.plot(e,chi2(2,scale=1/2).cdf(e),color='gray',lw=0.5)


#vcdf, = ax3.plot([],[],color='blue')
#ax3.set_xlim(0,3)
#ax3.set_ylim(-0.05,1.05)
#vv = np.linspace(0,5,200);

#ax3.plot(vv,chi(2,scale=np.sqrt(Em/m[0])).cdf(vv),color='gray',lw=0.5);
#ax3.plot(vv,chi(3,scale=sigma).cdf(vv),color='gray',ls='dashed');


dr = [plt.Circle(z[k][:2],R[k],color=colores[k%len(colores)], alpha=0.5) for k in range(n)]

for c in dr:
    ax.add_patch(c)

In [ ]:
dt = 0.01

def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                choque(i,j)


    z += v*dt


    for k in range(n):
        #dr[k].center=z[k][:2]
        dr[k].set_radius(0.01 + 0.03*z[k][2])


    E = np.sum(v**2,axis=1)/2 * m    
    cdf.set_data(sorted(E/Em),np.linspace(1/n,1,n))

    return []


fps = 30
T   = 1

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)

## 3D reparto

In [ ]:
from scipy.stats import norm, chi2, chi
import numpy.linalg as la

z = np.array([[x,y,z] for y in [0.6, 0.7, 0.8, 0.9] 
                        for x in np.linspace(0.1,0.9,10)
                        for z in [0.25,0.5,0.75]])
n = z.size//3

v = np.zeros([n,3])
v[:] = (1,0,0)

v[:] += 0.2*np.random.randn(n,3) 


R = 0.03*np.ones([n,1])

m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

frac = []
rep = []

In [ ]:
dt = 0.01



def choque(j,k,f=1):
    global v, frac
    g = z[j]-z[k]
    n = g/la.norm(g)

    wj = n@v[j]
    VJ = v[j] - wj*n

    wk = n@v[k]
    VK = v[k] - wk*n

    frac += [wj**2/la.norm(v[j])**2, wk**2/la.norm(v[k])**2]


    A = m[j]*wj + m[k]*wk
    M = m[j] + m[k]
    d = wk-wj

    nj = f*(A + m[k]*d) / M
    nk = f*(A - m[j]*d) / M

    v[j] = nj*n + VJ
    v[k] = nk*n + VK



def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                E0i = m[i]*(v[i]@v[i])/2
                E0j = m[j]*(v[j]@v[j])/2
                choque(i,j)
                E1i = m[i]*(v[i]@v[i])/2
                E1j = m[j]*(v[j]@v[j])/2
                rep.append([E0i/(E0i+E0j), E1i/(E1i+E1j)])


    z += v*dt

frac = []
rep  = []

for k in range(15*30):
    animate(k)


E = np.sum(v**2,axis=1)/2 * m    
plt.plot(sorted(E/Em),np.linspace(1/n,1,n));
e = np.linspace(0,5,200)
plt.plot(e,chi2(3,scale=1/3).cdf(e),color='gray',ls='dashed');

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(np.arange(1,1+len(frac))/len(frac),sorted(frac))
plt.plot(np.arange(1,1+len(frac))/len(frac),sorted(np.array(frac)**(1/2)),color='gray',lw=1)
plt.plot([0,1],[0,1],color='gray', ls='dashed')
plt.axis('equal');

In [ ]:
np.mean(frac)

In [ ]:
np.mean(np.array(frac)**(1/2))

In [ ]:
rep = np.array(rep)
plt.figure(figsize=(5,5))
plt.plot(rep[:,0],rep[:,1],'.',alpha=0.3);
plt.grid();

In [ ]:
len(rep)

En 2D. 

In [ ]:
z = np.array([[x,y] for y in [0.6, 0.7, 0.8, 0.9] for x in np.linspace(0.1,0.9,10)])
n = z.size//2

v = np.zeros([n,2])
v[:] = (1,0)

v[:] += 0.2*np.random.randn(n,2) 


R = 0.02*np.ones([n,1])


m = (R**3).flatten()

Et = sum(np.sum(v**2,axis=1)/2 * m)
Em = Et/n

In [ ]:
dt = 0.01

rep = []
frac = []

def animate(k):
    global z, v

    zs = z + v*dt
    v [ (zs > 1-R) | (zs < R) ]  *= -1

    for i in range(n-1):
        for j in range(i+1,n):
            if np.linalg.norm(zs[i] - zs[j]) < R[i]+R[j]:
                E0i = m[i]*(v[i]@v[i])/2
                E0j = m[j]*(v[j]@v[j])/2
                choque(i,j)
                E1i = m[i]*(v[i]@v[i])/2
                E1j = m[j]*(v[j]@v[j])/2
                rep.append([E0i/(E0i+E0j), E1i/(E1i+E1j)])


    z += v*dt


for k in range(60*30):
    animate(k)


E = np.sum(v**2,axis=1)/2 * m    
plt.plot(sorted(E/Em),np.linspace(1/n,1,n));
e = np.linspace(0,5,200)
plt.plot(e,chi2(2,scale=1/2).cdf(e),color='gray',ls='dashed');

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(np.arange(1,1+len(frac))/len(frac),sorted(frac))
plt.plot(np.arange(1,1+len(frac))/len(frac),sorted(np.array(frac)**(1/2)),color='gray',lw=1)
plt.plot([0,1],[0,1],color='gray', ls='dashed')
plt.axis('equal');

In [ ]:
np.mean(frac)

In [ ]:
rep = np.array(rep)
plt.figure(figsize=(5,5))
plt.plot(rep[:,0],rep[:,1],'.',alpha=0.3);
plt.grid();

In [ ]:
len(rep)

Hmm... curioso. No sé si estamos midiendo lo que queremos.

## Boltzmann

Cuando entidades que tienen energía media $E$ (o energía total $nE$) se emparejan al azar y se reparten su energía también al azar se alcanza la distribución de equilibrio de [Boltzmann](https://en.wikipedia.org/wiki/Boltzmann_distribution).

$$ \bar x = E $$

$$p(x) = \frac{1}{E} \exp\left( \frac{-x}{E}\right)$$

$$P(x) = 1 - \exp\left( \frac{-x}{E}\right)$$

Es muy fácil simular el proceso y comprobar que se alcanza la distribución de equilibrio.

In [ ]:
n = 1000
Em = 2

E = Em*np.ones(n)

T = 100*n

for _ in range(T):
    i = np.random.randint(n)
    j = np.random.randint(n)
    if i!= j:
        s = E[i]+E[j]
        a = s*np.random.rand()
        E[i] = a
        E[j] = s-a

plt.plot(sorted(E),(1+np.arange(n))/n);
e = np.linspace(0,E.max(),200);
plt.plot(e,1-np.exp(-e/Em));
plt.vlines(x=Em,ymin=0,ymax=1,linestyles='dashed',color='gray',lw=1);
plt.title('Maxwell-Boltzmann CDF'); plt.xlabel('$E$'); plt.ylabel('$P(x \leq E)$'); 

In [ ]:
Emax = 5*Em
bins = 20
ΔE = Emax/bins

h,b = np.histogram(E,range=(0,Emax),bins=bins)
b = (b[1:] + b[:-1])/2
h = h/n/ΔE
plt.bar(b,h,width=ΔE,edgecolor='black');

e = np.linspace(0,Emax,100)
f = np.exp(-e/Em) / Em

plt.plot(e,f,lw=3,color='red');
plt.vlines(x=Em,ymin=0,ymax=f[0],linestyles='dashed',color='gray')
plt.title('Maxwell-Boltzmann PDF'); plt.xlabel('$E$'); plt.ylabel('$P(E)$'); 

Para que el histograma quede bien la acumulada queda demasiado bien...

Veamos la evolución hacia el equilibrio. Hay 1000 cosas inicialmente ordenadas, todas con energía 1, que empiezan a interaccionar a un ritmo de 300 intercambios por segundo:

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.close();

n = 1000
Em = 1

E = Em*np.ones(n)

Emax = 5*Em
bins = 20
ΔE = Emax/bins

e = np.linspace(0,Emax,100)
f = np.exp(-e/Em) / Em


def animate(k):
    global E

    ax.clear()
    ax.set_xlim(-0.1,Emax)
    ax.set_ylim(0,1)
    h,b = np.histogram(E,range=(0,Emax),bins=bins)
    b = (b[1:] + b[:-1])/2
    ax.bar(b,h,width=ΔE,edgecolor='black');
    ax.set_xlim(0,Emax)
    ax.set_ylim(0,1100 if k < 4*30 else 250)
    ax.set_title('Boltzmann distribution'); ax.set_xlabel('E'); ax.set_ylabel('N(E)');
    if k> 10*30:
        ax.plot(e,n*Em*f*ΔE,lw=1,color='red');

    for _ in range(10):
        i = np.random.randint(n)
        j = np.random.randint(n)
        if i!= j:
            s = E[i]+E[j]
            a = s*np.random.rand()
            E[i] = a
            E[j] = s-a

    return []


fps = 30
T   = 1 #15
animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xticks(np.arange(0,100+1,20), minor=False)
ax.set_yticks(np.arange(0,100+1,20), minor=False)

ax.set_xticks(np.arange(0,100+1,4), minor=True)
ax.set_yticks(np.arange(0,100+1,4), minor=True)


ax.xaxis.grid(True, which='major',lw=2)
ax.xaxis.grid(True, which='minor',lw=0.5)
ax.yaxis.grid(True, which='major',lw=2)
ax.yaxis.grid(True, which='minor',lw=0.5)


plt.plot(100*np.arange(1,n+1)/n, 100*np.cumsum(sorted(E,reverse=True))/ (Em*n), lw=2);
plt.xlabel('rank (%)'); plt.ylabel('cumulative E (%)');

O sea, que el 20% más rico acapara el 50% de la riqueza. No tanto como en una power law.

Ahora bien, en el caso de la energía cinética de un gas esta distribución de energía se consigue en un mundo 2D, donde esta exponencial es equivalente a una $\chi^2(2)$. En 3D la densidad de estados da lugar a una $\chi^2(3)$, que surge de un intercambio parcial de energía, equivalente a jugarte una proporción $\alpha$. Experimentalmente hemos visto que las colisiones en 3D se reservan en promedio en 15% de su energía.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.close();

n = 1000
Em = 1

E = Em*np.ones(n)

Emax = 4*Em
bins = 30
ΔE = Emax/bins

e = np.linspace(0,Emax,100)
f = chi2(8,scale=1/8/Em).pdf(e)

def animate(k):
    global E

    ax.clear()
    ax.set_xlim(-0.1,Emax)
    ax.set_ylim(0,1)
    h,b = np.histogram(E,range=(0,Emax),bins=bins)
    b = (b[1:] + b[:-1])/2
    ax.bar(b,h,width=ΔE,edgecolor='black');
    ax.set_xlim(0,Emax)
    ax.set_ylim(0,1100 if k < 4*30 else 175)
    #ax.set_title('Boltzmann distribution');
    ax.set_xlabel('E'); ax.set_ylabel('N(E)');
    if k > 10*30:
        ax.plot(e,n*Em*f*ΔE,lw=1,color='red');

    for _ in range(10):
        i = np.random.randint(n)
        j = np.random.randint(n)
        if i!= j:
            s = E[i]/2+E[j]/2
            a = s*np.random.rand()
            E[i] = E[i]/2 + a
            E[j] = E[j]/2 + s-a

    return []


fps = 30
T   = 1 #15
animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.close();

n = 1000
Em = 1

E = Em*np.ones(n)

Emax = 4*Em
bins = 30
ΔE = Emax/bins

e = np.linspace(0,Emax,100)
f = chi2(3,scale=1/3/Em).pdf(e)

alpha = 6/7

def animate(k):
    global E

    ax.clear()
    ax.set_xlim(-0.1,Emax)
    ax.set_ylim(0,1)
    h,b = np.histogram(E,range=(0,Emax),bins=bins)
    b = (b[1:] + b[:-1])/2
    ax.bar(b,h,width=ΔE,edgecolor='black');
    ax.set_xlim(0,Emax)
    ax.set_ylim(0,1100 if k < 4*30 else 120)
    ax.set_title('Maxwell-Boltzmann distribution');
    ax.set_xlabel('E'); ax.set_ylabel('N(E)');
    if k > 10*30:
        ax.plot(e,n*Em*f*ΔE,lw=1,color='red');

    for _ in range(10):
        i = np.random.randint(n)
        j = np.random.randint(n)
        if i!= j:
            s = (E[i]+E[j])*alpha
            a = s*np.random.rand()
            E[i] = E[i]*(1-alpha) + a
            E[j] = E[j]*(1-alpha) + s-a

    return []


fps = 30
T   = 1 #20
animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True)

In [ ]:
%%time

alpha = 0.5
E[:] = 1

for _ in range(10*20*30):
        i = np.random.randint(n)
        j = np.random.randint(n)
        if i!= j:
            s = (E[i]+E[j])*alpha
            a = s*np.random.rand()
            E[i] = E[i]*(1-alpha) + a
            E[j] = E[j]*(1-alpha) + s-a

In [ ]:
plt.plot(e,chi2(8,scale=1/8).cdf(e))
plt.plot(sorted(E),np.arange(1,n+1)/n);

Parece que la proporción de energía que se pone en juego en 3D está cerca del 85%, frente al 100% en 2D.

Me sale algo parecido a lo siguiente, pero no encuentro una regla clara. Con $\alpha$ pequeño se va a un dof grande, que se aproxima a una gaussiana.

    dof:    2  3     4     5        8
    alpha:  1  0.85  3/4   2/3      1/2

Investigando un poco encontramos [un paper](https://arxiv.org/pdf/0905.1518.pdf) sobre *econophysics*, donde intenta modelar la distribución de dinero, riqueza, desigualdad, etc. con modelos de física estadística. Resulta que el proceso de jugarse en la interacción una fracción ya se ha estudiado y han encontrado la relación entre el factor de retención $\lambda = 1 - \alpha$ nuestro y los grados de libertad de la $\chi^2(k)$. El resultado, se simplifica a

$$\lambda = \frac{k-2}{k+4}$$

que cuadra con lo que habíamos encontrado empíricamente y además nos ajusta perfectamente el caso de 3 dof.

    k     :   1   2   3    4    5    6    7     8    9    10
    lambda: -1/5  0  1/7  1/4  1/3  2/5  5/11  1/2  7/13  4/7

Así pues, la densidad de estados 3D, o degeneración, corresponde a guardarte en promedio la proporción de energía 1/7 = {{1/7}}, muy próximo al valor empírico. Repetimos el vídeo con este valor.

Otro modelo de interacción discreto con el mismo resultado consiste en elegir un donante y receptor al azar y mover una unidad.

In [ ]:
n = 50
Em = 10
E = Em*np.ones(n)

In [ ]:
for _ in range(100*n):
    i = np.random.randint(n)
    j = np.random.randint(n)
    if i!= j and E[i] > 0:
        E[i] -= 1
        E[j] += 1

plt.hist(E,bins=20,range=(0,40));

En principio sabemos que un estado con energía $x$ debe tener una probabilidad **proporcional** a $\exp(-x a)$. Esto es debido a que "conseguir" una energía $x_1+x_2$ debe ser igual de probable que conseguir $x_1$ e independientemente también $x_2$, lo que implica que la probabilidad de la suma debe ser el producto de las probabilidades individuales. Y esto solo lo cumple la exponencial.

Pero la probabilidad (o densidad) exacta requiere normalizar. Y para ello ¡hay que saber cuántos estados hay con una cierta energía! No puedes suponer que todas tienen el mismo peso. En el ejemplo anterior, cada energía solo se puede tener de una manera, es directamente una propiedad de la partícula. La normalización sale directamente y es la energía media.

Pero cuando la energía se puede tener de diferentes maneras: p. ej., moviéndose a la misma velocidad en diferentes direcciones del espacio, entonces la *densidad de estados* sí cambia y hay que tenerlo en cuenta.

>All that is needed is to discover the density of microstates in energy, which is determined by dividing up momentum space into equal sized regions

De ahí la importancia de la función de partición, no es normalizar una función, sino saber cómo normalizarla dependiendo del sistema en cuestión.

Veamos la diferencia entre un sistema 2D y otro 3D.

Siempre tendremos

$$A \exp \left(\frac{-e}{E}\right) $$

$$e=\frac{p^2}{2m}\;\;\; p=\sqrt{2me}$$

$$de = \frac{p}{m} dp\;\; dp = \sqrt{\frac{m}{2e}}de $$

Las regiones que corresponden a un $de$:

$$dp^3 = 4 \pi p^2 dp$$

$$dp^2 = 2 \pi p dp$$

$$A \exp \left(\frac{-e}{E}\right) dp^3 = A \exp \left(\frac{-e}{E}\right) 4 \pi p^2 dp=$$

$$= A \exp \left(\frac{-e}{E}\right) 4 \pi (2me) \sqrt{\frac{m}{2e}}de\; \propto \exp \left(\frac{-e}{E}\right) \sqrt{e}\;de$$

La normalización me da:

$$p(e) = \frac{2}{\sqrt{\pi E^3}}\, \exp \left(\frac{-e}{E}\right) \sqrt{e} $$

Es una Chi-cuadrado de 3dof. Podemos comprobar en Wolfram Alpha o como queramos, que $p(e)$ está correctamente normalizada y que, curiosamente:

$$\int_0^\infty e p(e) de = \frac{3}{2}E$$

(Esto tengo que mirarlo bien). El tres puede venir de la media de la chi2(3), lo que da 1/2 de la energía media "de la otra" por dof, de acuerdo con equipartición de energía.

Para la velocidad...

Veamos qué ocurre en 2D:

$$A \exp \left(\frac{-e}{E}\right) dp^2 = A \exp \left(\frac{-e}{E}\right)2 \pi p dp=$$

$$= A \exp \left(\frac{-e}{E}\right) 2 \pi \sqrt{2me} \sqrt{\frac{m}{2e}}de\; \propto \exp \left(\frac{-e}{E}\right) \;de$$

La normalización me da:

$$p(e) = \frac{1}{E^2}\, \exp \left(\frac{-e}{E}\right) $$

Aquí la dependencia de $e$ que podría acompañar a la exponencial se cancela, y los 2 grados de libertad hacen que la densidad de estados sea constante. Queda igual que Boltzmann, con normalización y media igual a $E$. Es curioso, supongo. Es una chi2 con 2dof, escalada a la mitad o doble para cuadrar la media.

## Maxwell

Si las componentes de la velocidad en cada eje son independientes y su probabilidad solo depende de su magnitud nos sale una gaussiana multidimensional producto de 3 iguales. Este argumento es muy potente, el mismo que se utiliza para la distribución de los errores.

$$p(v_x) = \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{1}{2}\frac{v_x^2}{ \sigma^2} \right)$$

$$p(v_x,v_y,v_z) = \frac{1}{(\sqrt{2\pi} \sigma)^3} \exp\left(-\frac{1}{2}\frac{v_x^2 + v_y^2 + v_z^2}{ \sigma^2} \right)$$

En esféricas y marginalizando los ángulos:

$$p(v,\theta,\phi) \mathop{dV} = \frac{1}{(\sqrt{2\pi} \sigma)^3} \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right) \; v^2 \sin{\theta}\mathop {dv} \mathop{d\theta} \mathop{d\phi}$$

$$p(v) =  \frac{1}{(\sqrt{2\pi} \sigma)^3} \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right) v^2 \int_0^{2\pi} d\phi \int_0^{\pi} \sin(\theta)\mathop{d\phi}$$

$$p(v) =  \frac{4 \pi}{(\sqrt{2\pi} \sigma)^3}\,  v^2 \, \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right) $$

$$\bar v = 2 \sqrt{\frac{2}{\pi}} \sigma   \; \; \; \; \; \bar{v^2}= 3\sigma^2$$

Cambiando de variable para la energía cinética:

$$e = \frac{1}{2} m v^2 \; \; \; \; v = \sqrt{\frac{2e}{m}}\; \; \; \; dv = \frac{1}{\sqrt{2me}}\mathop{de} $$


$$p(e) =  \frac{4 \pi}{(\sqrt{2\pi} \sigma)^3}\,  \frac{2e}{m} \, \exp\left(-\frac{1}{2}\frac{ \frac{2e}{m}}{ \sigma^2} \right) \frac{1}{\sqrt{2me}}  =  \frac{2}{\sqrt{\pi}\sigma^3}   \frac{1}{m^{\frac{3}{2}}} \, \exp\left(-\frac{1}{2}\frac{ e}{ \frac{1}{2} m \sigma^2} \right) \sqrt{e} = $$

$$ = \frac{1}{\sqrt{2 \pi}\left( \frac{1}{2}m\sigma^2 \right)^\frac{3}{2}} \, \exp\left(-\frac{1}{2}\frac{ e}{ \frac{1}{2} m \sigma^2} \right) \sqrt{e} $$

$$\bar e = \int_0^\infty e p(e) de = \frac{3}{2} m\sigma^2$$

En 2D:

$$p(v_x,v_y,v_z) = \frac{1}{(\sqrt{2\pi} \sigma)^2} \exp\left(-\frac{1}{2}\frac{v_x^2 + v_y^2}{ \sigma^2} \right)$$

$$p(v,\theta) \mathop{dS} = \frac{1}{(\sqrt{2\pi} \sigma)^2} \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right) \; v  \sin{\theta}\mathop {dv} \mathop{d\theta} $$

$$p(v)  = \frac{2\pi}{(\sqrt{2\pi} \sigma)^2} \,  v \, \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right) = \frac{1}{\sigma^2} \,  v \, \exp\left(-\frac{1}{2}\frac{v^2}{ \sigma^2} \right)$$

$$\bar v = \sqrt{\frac{\pi}{2}} \sigma   \; \; \; \; \; \bar{v^2}= 2\sigma^2$$

La energía en este caso es:

$$p(e)  = \frac{1}{\sigma^2} \,  \,  \sqrt{\frac{2e}{m}}\, \exp\left(-\frac{1}{2}\frac{ \frac{2e}{m}}{ \sigma^2} \right) \frac{1}{\sqrt{2me}} = \frac{1}{2 \frac{1}{2} m \sigma^2} \, \exp\left(-\frac{1}{2}\frac{ e}{ \frac{1}{2} m \sigma^2} \right)$$

$$\bar e = \int_0^\infty e p(e) de = \frac{2}{2} m\sigma^2$$

Viendo la expresión para $\bar {v^2}$ de teoría cinética del gas ideal podemos expresar la variabilidad $\sigma$ de la velocidad en cada dimensión con K y T. Pero sobre todo, podemos relacionarla con la energía media (y total).

En 3D es

$$\sigma^2 = \frac{KT}{m} = \frac{2}{3}\frac{\bar e}{m}$$

Y en 2D:

$$\sigma^2 = \frac{KT}{m} = \frac{\bar e}{m}$$

Es muy interesante: hacemos una simulación de partículas en una caja con choques elásticos, poniendo las velocidades inciales que queramos. Por tanto, tenemos la energía media y con ella completamente caracterizada la distribución de velocidades, cuando llegue al equilibrio, lo cual normalmente sucederá pronto.

Entonces podemos predecir la velocidad media (y todo lo demás), a partir de es simple $\sigma$ que la caracteriza. Pero ojo, no es trivial: ¡depende de la dimensión del espacio!

Una posible forma de pensar en la entropía: La distribución de energía y de $\bar{v^2}$, aunque va cambiando, típicamente desde una configuración ordenada hasta el equilibrio, siempre tiene la misma media. Pero las de las v magnitud o marginal no.

La diferencia entre 2D y 3D para la misma energía media-total no es mucha pero se nota:

In [ ]:
e = np.linspace(0,5,100)
plt.plot(e,chi2(2,scale=1/2).cdf(e),label='2dof / Boltmann');
plt.plot(e,chi2(3,scale=1/3).cdf(e),label='3dof / Maxwell-Boltmann');
plt.legend();

In [ ]:
e = np.linspace(0,5,100)
plt.plot(e,chi2(2,scale=1/2).pdf(e),label='2dof / Boltmann');
plt.plot(e,chi2(3,scale=1/3).pdf(e),label='3dof / Maxwell-Boltmann');
plt.legend();